In [2]:
import pandas as pd
import re
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.model_selection import train_test_split
from pathlib import Path
import gc
import glob
import os

In [3]:
tqdm.pandas()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
path = Path("C:/Users/hp/Desktop/depi_final_pro/writingPrompts/")
files = ["train.wp_source", "train.wp_target", "valid.wp_source", "valid.wp_target", "test.wp_source", "test.wp_target"]

In [5]:
print(" Loading all writingPrompts files...")

data = []
for f in files:
    fpath = path / f
    if not fpath.exists():
        print(" Missing:", fpath)
        continue
    with open(fpath, "r", encoding="utf-8") as file:
        lines = file.readlines()
    print(f"Loaded {len(lines)} lines from {f}")
    data.append(lines)

 Loading all writingPrompts files...
Loaded 272600 lines from train.wp_source
Loaded 272600 lines from train.wp_target
Loaded 15620 lines from valid.wp_source
Loaded 15620 lines from valid.wp_target
Loaded 15138 lines from test.wp_source
Loaded 15138 lines from test.wp_target


In [6]:
sources = []
targets = []
for i, f in enumerate(files):
    if "source" in f:
        sources.extend(data[i])
    elif "target" in f:
        targets.extend(data[i])

print(" Total prompts:", len(sources))
print("Total responses:", len(targets))


 Total prompts: 303358
Total responses: 303358


In [7]:
min_len = min(len(sources), len(targets))
df = pd.DataFrame({
    "prompt": sources[:min_len],
    "response": targets[:min_len]
})

In [8]:
def clean_text(text):
    text = str(text)
    text = re.sub(r'<[^>]+>', ' ', text)  # HTML
    text = re.sub(r'http\S+|www\.\S+', ' ', text)  # URLs
    text = re.sub(r'[^0-9A-Za-z\u0600-\u06FF\s\.\,\?\!\:\;\'\"\-\(\)\[\]]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

print("Cleaning texts...")
df['clean_text'] = df['response'].progress_apply(clean_text)
df = df.dropna(subset=['clean_text']).reset_index(drop=True)
df = df.drop_duplicates(subset=['clean_text']).reset_index(drop=True)

print(" Cleaned rows:", len(df))

Cleaning texts...


100%|██████████| 303358/303358 [06:01<00:00, 838.59it/s] 


 Cleaned rows: 303072


In [9]:

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def process_batch(batch_df, start_index):
    batch_result = []
    for idx, row in batch_df.iterrows():
        text = row['clean_text']

        # Tokenize
        words = word_tokenize(text)
        sents = sent_tokenize(text)

        # Remove stopwords
        words = [w for w in words if w.lower() not in stop_words]

        # Lemmatize
        words = [lemmatizer.lemmatize(w, pos='v') for w in words]

        processed_text = ' '.join(words)
        batch_result.append({
            'clean_text': text,
            'processed_text': processed_text,
            'n_words': len(words),
            'n_sentences': len(sents)
        })

    result_df = pd.DataFrame(batch_result)
    result_df.to_csv(f"temp_batch_{start_index}.csv", index=False)
    del batch_result
    gc.collect()

BATCH_SIZE = 5000  
total_rows = len(df)
print(f" Processing {total_rows} rows in batches of {BATCH_SIZE}")

for start in tqdm(range(0, total_rows, BATCH_SIZE)):
    end = min(start + BATCH_SIZE, total_rows)
    batch_df = df.iloc[start:end]
    print(f"Processing batch {start} → {end}")
    process_batch(batch_df, start)
    del batch_df
    gc.collect()

print(" All batches processed!")

 Processing 303072 rows in batches of 5000


  0%|          | 0/61 [00:00<?, ?it/s]

Processing batch 0 → 5000


  2%|▏         | 1/61 [01:30<1:30:41, 90.70s/it]

Processing batch 5000 → 10000


...

Processing batch 295000 → 300000


 98%|█████████▊| 60/61 [1:15:28<00:59, 59.63s/it]

Processing batch 300000 → 303072


100%|██████████| 61/61 [1:16:06<00:00, 74.86s/it]

 All batches processed!


In [11]:
all_batches = glob.glob("temp_batch_*.csv")
df_list = [pd.read_csv(f) for f in all_batches]
final_df = pd.concat(df_list, ignore_index=True)

final_df.to_csv("full_clean.csv", index=False)
print(" Saved full_clean.csv with", len(final_df), "rows")

 Saved full_clean.csv with 303072 rows


In [12]:
for f in all_batches:
    os.remove(f)
print("Temporary files deleted.")

Temporary files deleted.


In [13]:
import os
print(" Current working directory:", os.getcwd())


 Current working directory: c:\Users\hp\Desktop\depi_final_pro


In [14]:
train_df, test_df = train_test_split(final_df[['clean_text', 'processed_text']], test_size=0.2, random_state=42)
train_df.to_csv("train_clean.csv", index=False)
test_df.to_csv("test_clean.csv", index=False)

print(" Done! Saved:")
print(" train_clean.csv")
print(" test_clean.csv")
print(" full_clean.csv")

 Done! Saved:
 train_clean.csv
 test_clean.csv
 full_clean.csv


In [ ]:
import pandas as pd

train_df = pd.read_csv("train_clean.csv")
test_df  = pd.read_csv("test_clean.csv")

train_text = train_df["processed_text"].astype(str)
test_text  = test_df["processed_text"].astype(str)

print("Data Loaded Successfully")

Data Loaded Successfully ✔


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X_train = tfidf.fit_transform(train_text).toarray()
X_test  = tfidf.transform(test_text).toarray()

print("TF-IDF Vectorization Done ")

TF-IDF Vectorization Done ✔


In [ ]:
!pip install transformers datasets accelerate sentencepiece evaluate

import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

#  تحميل بيانات التدريب والاختبار
train_df = pd.read_csv("train_clean.csv")
test_df  = pd.read_csv("test_clean.csv")

train_dataset = Dataset.from_pandas(train_df[["processed_text"]])   # ← أهم عمود
test_dataset  = Dataset.from_pandas(test_df[["processed_text"]])
print("Dataset Loaded Successfully ")

     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.0 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.0 kB 217.9 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/44.0 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 44.0/44.0 kB 239.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/77.6 kB ? eta -:--:--
     ------------------------------------ --- 71.7/77.6 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 77.6/77.6 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.0 MB 2.2 MB/s eta 0:00:06
    --------------------------------------- 0.2/12.0 MB 2.1 MB/s eta 0:00:06
    --------------------------------------- 0.3/12.0 MB 2.0 MB/s eta 0:00:06
   - ----


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset Loaded Successfully ✔


In [ ]:
# إصلاح القيم الغير نصية وحذف الفارغ
train_df["processed_text"] = train_df["processed_text"].astype(str)
test_df["processed_text"] = test_df["processed_text"].astype(str)

train_df = train_df.dropna(subset=["processed_text"])
test_df = test_df.dropna(subset=["processed_text"])

#  إزالة النصوص الفاضية أو اللي فيها مسافات فقط
train_df = train_df[train_df["processed_text"].str.strip() != ""]
test_df = test_df[test_df["processed_text"].str.strip() != ""]

#  إعادة بناء الـ Dataset بعد التنظيف
train_dataset = Dataset.from_pandas(train_df[["processed_text"]])
test_dataset = Dataset.from_pandas(test_df[["processed_text"]])

print("Dataset Fixed  All Good for Tokenization")


Dataset Fixed ✔ All Good for Tokenization!


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  

def tokenize(batch):
    return tokenizer(batch["processed_text"],
                     padding="max_length",
                     truncation=True,
                     max_length=128)

train_tokenized = train_dataset.map(tokenize, batched=True)
test_tokenized  = test_dataset.map(tokenize, batched=True)

print("Tokenization Done")


Map: 100%|██████████| 60615/60615 [05:30<00:00, 183.44 examples/s]

Tokenization ✔ Done


In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

#  تحميل نموذج GPT-2 الأساسي
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

#  Collator لضبط البيانات أثناء التدريب
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT-2 مش Masked Language Model
)

#  إعدادات التدريب
training_args = TrainingArguments(
    output_dir="./creative-gpt2",
    overwrite_output_dir=True,
    num_train_epochs=1,       
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="epoch",
    logging_steps=50,
    save_steps=200,
    warmup_steps=50,
    weight_decay=0.01,
)

#  Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
)

print(" Training Started...")
trainer.train()
print(" Training Finished!")


In [ ]:
import math

eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")


In [ ]:
prompt = "اكتب لي قصة قصيرة عن النجاح"
inputs = tokenizer(prompt, return_tensors="pt")

output = model.generate(
    **inputs,
    max_length=100,
    num_beams=5,
    no_repeat_ngram_size=2,
    temperature=0.9,
    top_p=0.95,
    do_sample=True
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


In [11]:
!pip install gradio

import gradio as gr

def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(
        **inputs,
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=2,
        temperature=0.9,
        top_p=0.95,
        do_sample=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

app = gr.Interface(
    fn=generate_text,
    inputs="text",
    outputs="text",
    title="AI Creative Writing Assistant",
    description="اكتب جملة وسيقوم النموذج بإكمال النص الإبداعي تلقائيًا"
)

app.launch()



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
     -------------------------------------- 42.9/42.9 kB 695.5 kB/s eta 0:00:00
     ---------------------------------------- 0.0/89.9 kB ? eta -:--:--
     ------------------------------- -------- 71.7/89.9 kB 2.0 MB/s eta 0:00:01
     ------------------------------------ --- 81.9/89.9 kB 1.2 MB/s eta 0:00:01
     ---------------------------------------- 89.9/89.9 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/21.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/21.6 MB 3.4 MB/s eta 0:00:07
   ---------------------------------------- 0.1/21.6 MB 2.1 MB/s eta 0:00:10
   ---------------------------------------- 0.2/21.6 MB 2.3 MB/s eta 0:00:10
    --------------------------------------- 0.3/21.6 MB 2.1 MB/s eta 0:00:11
    --------------------------------------- 0.4/21.6 MB 2.0 MB/s eta 0:00:11
    --------------------------------------- 0.5/21.6 MB 2.0 MB/s eta 0:00:11
   -

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Created dataset file at: .gradio\flagged\dataset1.csv


In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

prompt = "اكتب شعر عن النجاح"
inputs = tokenizer(prompt, return_tensors="pt")

output = model.generate(
    **inputs,
    max_length=120,
    do_sample=True,
    top_p=0.95,
    temperature=0.8
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


اكتب شعر عن النجاحة شدكون المحمد الصور بن باكتب شعر عن النجاحة شدكون المحمد الصور بن باكتب شعر عن النجاحة شدكون المحمد الصو


In [ ]:
import math
from datasets import Dataset

eval_dataset = Dataset.from_pandas(test_df[["processed_text"]].sample(50))  

inputs = tokenizer(eval_dataset[0]["processed_text"], return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
print("Perplexity:", math.exp(loss))


Perplexity: 297.724316486039


C:\Users\hp\AppData\Local\Temp\ipykernel_16912\1809933616.py:9: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:837.)
  print("Perplexity:", math.exp(loss))


In [14]:
!pip install gradio

import gradio as gr

def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(
        **inputs, max_length=100, do_sample=True,
        top_p=0.95, temperature=0.8
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

app = gr.Interface(
    fn=generate_text,
    inputs="text",
    outputs="text",
    title="AI Creative Writing Assistant"
)

app.launch()



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
